In [2]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance

from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Import Qiskit libraries for VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA, COBYLA

# Import Qiskit Nature libraries
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.circuit.library import UCC, UCCSD
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


from qiskit.primitives import Estimator
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock


In [3]:

from qiskit.primitives import Estimator, BackendEstimator

from qiskit.algorithms import VQE
from qiskit_nature.units import DistanceUnit
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

In [4]:
bond_distance=0.735
molecule = f"H 0 0 0; H 0 0 {bond_distance}"
driver = PySCFDriver(atom=molecule,charge=0,spin=0,basis='sto3g',unit=DistanceUnit.ANGSTROM,)
# run the deriver
es_problem = driver.run()

In [5]:
second_q_ops = es_problem.second_q_ops()
num_orbitals = es_problem.num_spin_orbitals
num_particles = es_problem.num_particles

In [7]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

fermionic_op = es_problem.hamiltonian.second_q_op()
#choose the mapper 
mapper = JordanWignerMapper() 
#mapper = BravyiKitaevMapper() 
qubit_op = mapper.map(fermionic_op)
print(qubit_op)

-0.8105479805373283 * IIII
+ 0.17218393261915543 * IIIZ
- 0.2257534922240237 * IIZI
+ 0.12091263261776633 * IIZZ
+ 0.17218393261915543 * IZII
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
- 0.22575349222402363 * ZIII
+ 0.1661454325638241 * ZIIZ
+ 0.16614543256382408 * IZZI
+ 0.1746434306830045 * ZIZI
+ 0.12091263261776633 * ZZII


In [8]:
backend = Aer.get_backend('statevector_simulator')
qinstance = QuantumInstance(backend, shots=1024, seed_simulator=2, seed_transpiler=2)

In [ ]:
qaoa = QAOA(optimizer=COBYLA(), quantum_instance=qinstance)
result = qaoa.compute_minimum_eigenvalue(qubit_op)

In [10]:
print(result.eigenvalue.real)

-1.1122711181328957


In [12]:
qaoa = QAOA(optimizer=COBYLA(maxiter=100), quantum_instance=qinstance)
result = qaoa.compute_minimum_eigenvalue(qubit_op)
print(result.eigenvalue.real)

-1.1122711018367493
